In [67]:
import numpy as np
import pyphi

from qutip import *
from qutip_qip.operations import *
from intrinsic_difference import intrinsic_difference
from utils import *
from compute_ces import *

In [68]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Define states and matrices

In [53]:
# Basis states (for both classical and quantum)
zero = basis(2,0)
one = basis(2,1)

# Normalized fully general quantum state
plus = (zero + one).unit()
minus = (zero - one).unit()

# Density matrix representations of the states
rho_zero = zero * zero.dag() # could also be fock_dm(2,0)
rho_one = one * one.dag()
rho_plus = plus * plus.dag()
rho_minus = minus * minus.dag()
rho_mm = 0.5 * rho_zero + 0.5 * rho_one

rho_bell = ket2dm(bell_state(state = '00'))
rho_GHZ = ket2dm(ghz_state(N=3))
rho_W = ket2dm(w_state(N=3))
rho_bellmix = 1/3*(tensor(rho_bell, rho_zero) + tensor(rho_zero, rho_bell) + tensor(rho_zero, rho_bell).permute([1,0,2]))

rho_bennet = 1/(3-4)*(1-(tensor(rho_zero, rho_one, rho_plus) + tensor(rho_one, rho_plus, rho_zero) 
                        + tensor(rho_plus, rho_zero, rho_one) + tensor(rho_minus, rho_minus, rho_minus)))


In [51]:
pyphi.config.MEASURE = 'ID'
pyphi.config.PARTITION_TYPE = 'ALL'
pyphi.config.PICK_SMALLEST_PURVIEW = False

In [52]:
print('ID |0> : ', intrinsic_difference(rho_zero, rho_mm))
print('ID |+> : ', intrinsic_difference(rho_plus, rho_mm))

ID |0> :  (1.0, [array([-1.+0.j, -0.+0.j])])
ID |+> :  (1.0, [array([0.70710678+0.j, 0.70710678+0.j])])


# 2 Qubits - CNOT

In [81]:
cnot()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]

## Classical results

In [82]:
print(pyphi.config)

{ 'ACTUAL_CAUSATION_MEASURE': 'PMI',
  'ASSUME_CUTS_CANNOT_CREATE_NEW_CONCEPTS': False,
  'CACHE_POTENTIAL_PURVIEWS': True,
  'CACHE_REPERTOIRES': True,
  'CACHE_SIAS': False,
  'CACHING_BACKEND': 'fs',
  'CLEAR_SUBSYSTEM_CACHES_AFTER_COMPUTING_SIA': False,
  'CUT_ONE_APPROXIMATION': False,
  'FS_CACHE_DIRECTORY': '__pyphi_cache__',
  'FS_CACHE_VERBOSITY': 0,
  'LOG_FILE': 'pyphi.log',
  'LOG_FILE_LEVEL': 'INFO',
  'LOG_STDOUT_LEVEL': 'WARNING',
  'MAXIMUM_CACHE_MEMORY_PERCENTAGE': 50,
  'MEASURE': 'ID',
  'MONGODB_CONFIG': { 'collection_name': 'cache',
                      'database_name': 'pyphi',
                      'host': 'localhost',
                      'port': 27017},
  'NUMBER_OF_CORES': -1,
  'PARALLEL_COMPLEX_EVALUATION': False,
  'PARALLEL_CONCEPT_EVALUATION': False,
  'PARALLEL_CUT_EVALUATION': True,
  'PARTITION_TYPE': 'ALL',
  'PICK_SMALLEST_PURVIEW': False,
  'PRECISION': 6,
  'PRINT_FRACTIONS': True,
  'PROGRESS_BARS': True,
  'REDIS_CACHE': False,
  'REDIS_CONFIG'

In [83]:
CNOT_tpm = cnot().data.toarray()
#CNOT_tpm = pyphi.convert.le2be_state_by_state(CNOT_tpm.real)

In [84]:
CNOT_tpm.real

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])

In [85]:
# pyphi convention for state ordering is little endian
CNOT_tpm = pyphi.convert.be2le_state_by_state(CNOT_tpm.real)

In [86]:
network = pyphi.Network(CNOT_tpm)
subsystem = pyphi.Subsystem(network, (1,0))

In [14]:
ces = pyphi.compute.ces(subsystem)

In [15]:
ces

════════════════════════════════════════════════════════════
            Cause-effect structure (2 concepts)             
════════════════════════════════════════════════════════════
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  
              Concept: Mechanism = [n0], φ = 1              
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  
              MIC                         MIE               
  ┌──────────────────────────┐┌──────────────────────────┐  
  │  φ = 1                   ││  φ = 1                   │  
  │  Purview = [n0]          ││  Purview = [n0]          │  
  │  MIP:                    ││  MIP:                    │  
  │    n0     ∅              ││    n0     ∅              │  
  │    ─── ✕ ───             ││    ─── ✕ ───             │  
  │     ∅    n0              ││     ∅    n0              │  
  │  Repertoire:             ││  Repertoire:             │  
  │    ┌─────────────┐       ││    ┌─────────────┐       │  
  │    │ S    Pr(S)  │  

## Quantum results

In [73]:
qgate = cnot()

In [74]:
#rho = 0.5*tensor(rho_one, rho_zero) + 0.5*tensor(rho_zero, rho_zero)
rho = tensor(rho_mm, rho_zero)

In [75]:
rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.5 0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.5 0. ]
 [0.  0.  0.  0. ]]

In [76]:
rho_e = evolve(rho, qgate, 'effect')
rho_e

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.5 0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0.5]]

In [77]:
result = compute_ces(rho, qgate, direction = 'effect')

m:  (0,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0
m:  (1,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0
m:  (0, 1)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0


In [78]:
result

[]

In [79]:
result_cause = compute_ces(rho_e, qgate, direction = 'cause')

m:  (0,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0
m:  (1,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0
m:  (0, 1)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0


In [80]:
result_cause

[]

In [162]:
from qutip.measurement import measurement_statistics
measurement_statistics(rho, tensor(rho_zero, rho_zero))

(array([0., 0., 0., 1.]),
 [Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
  Qobj data =
  [[0. 0. 0. 0.]
   [0. 1. 0. 0.]
   [0. 0. 0. 0.]
   [0. 0. 0. 0.]],
  Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
  Qobj data =
  [[0. 0. 0. 0.]
   [0. 0. 0. 0.]
   [0. 0. 1. 0.]
   [0. 0. 0. 0.]],
  Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
  Qobj data =
  [[0. 0. 0. 0.]
   [0. 0. 0. 0.]
   [0. 0. 0. 0.]
   [0. 0. 0. 1.]],
  Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
  Qobj data =
  [[1. 0. 0. 0.]
   [0. 0. 0. 0.]
   [0. 0. 0. 0.]
   [0. 0. 0. 0.]]],
 [0.0, 0.0, 0.5, 0.5])

In [152]:
sigmaz()

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[ 1.  0.]
 [ 0. -1.]]

In [159]:
Z0, Z1 = ket2dm(basis(2, 0)), ket2dm(basis(2, 1))

state_0 = basis(2, 0)

state_plus = (basis(2, 0) + basis(2, 1)).unit()

state_0plus = tensor(state_0, state_plus)

PZ1 = [tensor(Z0, identity(2)), tensor(Z1, identity(2))]

PZ2 = [tensor(identity(2), Z0), tensor(identity(2), Z1)]

collapsed_states, probabilities = measurement_statistics(state_0plus, PZ2)


In [160]:
collapsed_states

[Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
 Qobj data =
 [[1.]
  [0.]
  [0.]
  [0.]],
 Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
 Qobj data =
 [[0.]
  [1.]
  [0.]
  [0.]]]

In [161]:
probabilities

[0.4999999999999999, 0.4999999999999999]

# 3 Qubits

In [138]:
tensor(qeye(2), cnot())

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0.]]

In [57]:
entanglement_partition(rho_bellmix)

[(0, 1, 2)]

In [58]:
ent_partition = [(1,), (0,2)]

In [64]:
ent_indices = []
for i in ent_partition:
    ent_indices.extend(list(i))

sort_index = np.argsort(ent_indices)

print(sort_index)

[1 0 2]


In [63]:
ent_indices

[1, 0, 2]

In [65]:
np.argsort([0, 1, 5, 3, 2])

array([0, 1, 4, 3, 2])

In [91]:
decorrelate_rho(rho_GHZ, [(1,),(0,2)])

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0.25 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.25 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.25 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.25]]

In [84]:
len(toffoli().dims[0])

3

In [82]:
cnot

<function qutip_qip.operations.gates.cnot(N=None, control=0, target=1)>

In [88]:
part = [rho_zero]
for mm in range(2):
    part.append(rho_mm)
        

In [90]:
tensor(part)

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0.25 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.25 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.25 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.25 0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]]